# Sentiment Analysis using TF-Learn (Game)

In [1]:
import sys
import tensorflow as tf
import pandas as pd

C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
n_epoch = int(input('Enter number of epochs for the RNN training: '))

Enter number of epochs for the RNN training: 100


In [3]:
#Load IGN dataset
ign = pd.read_csv('ign.csv')
ign.head()

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12
3,3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11
4,4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11


In [4]:
#check for the shape of the dataset
ign.shape

(18625, 11)

In [5]:
#Check out all the unique score_phrase as well as their counts
ign.score_phrase.value_counts()

Great          4773
Good           4741
Okay           2945
Mediocre       1959
Amazing        1804
Bad            1269
Awful           664
Painful         340
Unbearable       72
Masterpiece      55
Disaster          3
Name: score_phrase, dtype: int64

# Data Preprocessing

In [6]:
bad_phrases = ['Bad', 'Awful', 'Painful', 'Unbearable', 'Disaster']
ign['sentiment'] = ign.score_phrase.isin(bad_phrases).map({True:'Negative', False:'Positive'})

In [7]:
#Remove Disaster rows since the number is very low (outlier)
ign = ign[ign['score_phrase'] != 'Disaster']

In [8]:
ign.head()

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day,sentiment
0,0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12,Positive
1,1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12,Positive
2,2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12,Positive
3,3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11,Positive
4,4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11,Positive


In [9]:
ign.score_phrase.value_counts()

Great          4773
Good           4741
Okay           2945
Mediocre       1959
Amazing        1804
Bad            1269
Awful           664
Painful         340
Unbearable       72
Masterpiece      55
Name: score_phrase, dtype: int64

In [10]:
#No. of Positive Sentiments VS No. of Negative Seniments
ign.sentiment.value_counts(normalize=True)

Positive    0.874074
Negative    0.125926
Name: sentiment, dtype: float64

In [11]:
#Check for null elements
ign.isnull().sum()

Unnamed: 0         0
score_phrase       0
title              0
url                0
platform           0
score              0
genre             36
editors_choice     0
release_year       0
release_month      0
release_day        0
sentiment          0
dtype: int64

In [12]:
ign[ign.genre.isnull()]

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day,sentiment
12,12,Good,Wild Blood,/games/wild-blood/iphone-139363,iPhone,7.0,NaN,N,2012,9,10,Positive
113,113,Good,Retro/Grade,/games/retrograde-138590/ps3-21766,PlayStation 3,7.0,NaN,N,2012,8,15,Positive
160,160,Good,10000000,/games/10000000/iphone-139135,iPhone,7.5,NaN,N,2012,8,9,Positive
176,176,Okay,Colour Bind,/games/colour-bind/pc-143757,PC,6.2,NaN,N,2012,10,15,Positive
9375,9375,Great,Duke Nukem Arena,/games/duke-nukem-arena/cell-893821,Wireless,8.0,NaN,Y,2007,6,15,Positive
9488,9488,Okay,Rengoku,/games/rengoku/cell-924924,Wireless,6.5,NaN,N,2007,6,26,Positive
9767,9767,Good,Super Sketcher,/games/super-sketcher/cell-874054,Wireless,7.5,NaN,N,2007,9,14,Positive
9774,9774,Amazing,Critter Crunch,/games/critter-crunch/cell-963486,Wireless,9.0,NaN,Y,2007,9,13,Positive
10494,10494,Awful,Clue / Mouse Trap / Perfection / Aggravation,/games/clue-mouse-trap-perfection-aggravation/...,Nintendo DS,3.5,NaN,N,2008,1,23,Negative
11367,11367,Painful,Jeep Thrills,/games/jeep-thrills/ps2-14246598,PlayStation 2,2.0,NaN,N,2008,8,18,Negative


In [13]:
#Fill all null elements with an empty string
ign.fillna(value='', inplace=True)

In [14]:
ign[ign.genre.isnull()]

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day,sentiment


In [15]:
#Create a new DataFrame

df_ign = ign[['sentiment', 'score_phrase', 'title', 'platform', 'genre', 'editors_choice']].copy()
df_ign.head()

,sentiment,score_phrase,title,platform,genre,editors_choice
0,Positive,Amazing,LittleBigPlanet PS Vita,PlayStation Vita,Platformer,Y
1,Positive,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,PlayStation Vita,Platformer,Y
2,Positive,Great,Splice: Tree of Life,iPad,Puzzle,N
3,Positive,Great,NHL 13,Xbox 360,Sports,N
4,Positive,Great,NHL 13,PlayStation 3,Sports,N


In [16]:
df_ign['is_editors_choice'] = df_ign['editors_choice'].map({'Y': 'editors_choice', 'N': ''})

In [17]:
#Create a new column called text which contains contents of several columns
df_ign['text'] = df_ign['title'].str.cat(df_ign['platform'], sep=' ').str.cat(df_ign['genre'], sep=' ').str.cat(df_ign['is_editors_choice'], sep=' ')
df_ign.head(10)

,sentiment,score_phrase,title,platform,genre,editors_choice,is_editors_choice,text
0,Positive,Amazing,LittleBigPlanet PS Vita,PlayStation Vita,Platformer,Y,editors_choice,LittleBigPlanet PS Vita PlayStation Vita Platf...
1,Positive,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,PlayStation Vita,Platformer,Y,editors_choice,LittleBigPlanet PS Vita -- Marvel Super Hero E...
2,Positive,Great,Splice: Tree of Life,iPad,Puzzle,N,,Splice: Tree of Life iPad Puzzle
3,Positive,Great,NHL 13,Xbox 360,Sports,N,,NHL 13 Xbox 360 Sports
4,Positive,Great,NHL 13,PlayStation 3,Sports,N,,NHL 13 PlayStation 3 Sports
5,Positive,Good,Total War Battles: Shogun,Macintosh,Strategy,N,,Total War Battles: Shogun Macintosh Strategy
6,Negative,Awful,Double Dragon: Neon,Xbox 360,Fighting,N,,Double Dragon: Neon Xbox 360 Fighting
7,Positive,Amazing,Guild Wars 2,PC,RPG,Y,editors_choice,Guild Wars 2 PC RPG editors_choice
8,Negative,Awful,Double Dragon: Neon,PlayStation 3,Fighting,N,,Double Dragon: Neon PlayStation 3 Fighting
9,Positive,Good,Total War Battles: Shogun,PC,Strategy,N,,Total War Battles: Shogun PC Strategy


In [28]:
df_ign.shape

(18622, 8)

# Here, I'll treat this as a multiclass problem where I attempt to predict the labels (i.e. the score_phrases)


In [18]:
X = df_ign.text
Y = df_ign.score_phrase

In [19]:
X.head(10)

0    LittleBigPlanet PS Vita PlayStation Vita Platf...
1    LittleBigPlanet PS Vita -- Marvel Super Hero E...
2                    Splice: Tree of Life iPad Puzzle 
3                              NHL 13 Xbox 360 Sports 
4                         NHL 13 PlayStation 3 Sports 
5        Total War Battles: Shogun Macintosh Strategy 
6               Double Dragon: Neon Xbox 360 Fighting 
7                   Guild Wars 2 PC RPG editors_choice
8          Double Dragon: Neon PlayStation 3 Fighting 
9               Total War Battles: Shogun PC Strategy 
Name: text, dtype: object

In [20]:
Y.head(10)

0    Amazing
1    Amazing
2      Great
3      Great
4      Great
5       Good
6      Awful
7    Amazing
8      Awful
9       Good
Name: score_phrase, dtype: object

# The DUMMY Classifier (Always Choose the Most Frequent Class)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [22]:
vect= TfidfVectorizer(stop_words='english', token_pattern=r'\b\w{2,}\b')
dummy = DummyClassifier(strategy='most_frequent', random_state=0)
dummy_pipeline = make_pipeline(vect, dummy)
dummy_pipeline.named_steps

{'dummyclassifier': DummyClassifier(constant=None, random_state=0, strategy='most_frequent'),
 'tfidfvectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words='english', strip_accents=None, sublinear_tf=False,
         token_pattern='\\b\\w{2,}\\b', tokenizer=None, use_idf=True,
         vocabulary=None)}

In [23]:
#Cross validation
cv = cross_val_score(dummy_pipeline, X, Y, scoring='accuracy', cv=10, n_jobs=-1)
print('Dummy Classifier accuracy: ', cv.mean())

Dummy Classifier accuracy:  0.2563100451112067


# MultinomialNB Classifier

In [24]:
from sklearn.naive_bayes import MultinomialNB

In [25]:
vect = TfidfVectorizer(stop_words='english', token_pattern=r'\b\w{2,}\b', min_df=1, max_df=0.1, ngram_range=(1,2))
mnb = MultinomialNB(alpha=2)
mb_pipeline = make_pipeline(vect, mnb)

In [26]:
mb_pipeline.named_steps

{'multinomialnb': MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
 'tfidfvectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.1, max_features=None, min_df=1,
         ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words='english', strip_accents=None, sublinear_tf=False,
         token_pattern='\\b\\w{2,}\\b', tokenizer=None, use_idf=True,
         vocabulary=None)}

In [29]:
#Cross validation
cv = cross_val_score(mb_pipeline, X, Y, scoring='accuracy', cv=10, n_jobs=-1)
print('Multimoulli Classifier accuracy: ', cv.mean())

Multimoulli Classifier accuracy:  0.3235488238541296


# RNN Classifier using TFLearn

In [50]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [62]:
#Create the vocab (so that we can create X_word_ids from X)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w{1,}\b')

In [104]:
vect.fit(X_train)
vocab = vect.vocabulary_

In [105]:
def convert_X_to_X_word_ids(X):
    return X.apply( lambda x: [vocab[w] for w in [w.lower().strip() for w in x.split()] if w in vocab] )

In [106]:
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids = convert_X_to_X_word_ids(X_test)

In [107]:
# Difference between X(_train/_test) and X(_train_word_ids/test_word_ids)
X_train.head()


16138    Castlevania: Harmony of Despair PlayStation 3 ...
5945     Kim Possible 2: Drakken's Demise Game Boy Adva...
11360                  Madden NFL 09 PlayStation 2 Sports 
18270                         WWE 2K16 Xbox One Wrestling 
12533              The Last Ninja Commodore 64/128 Action 
Name: text, dtype: object

In [108]:
X_train_word_ids.head()

16138                 [3134, 4717, 1911, 5074, 149, 269]
5945     [3730, 5126, 1888, 2799, 1025, 266, 5062, 2227]
11360                   [4037, 4585, 13, 5074, 77, 6257]
18270                      [7447, 136, 7458, 4751, 7439]
12533                      [6674, 3843, 4617, 1533, 251]
Name: text, dtype: object

In [110]:
print('X_train_word_ids.shape:', X_train_word_ids.shape)
print('X_test_word_ids.shape:', X_test_word_ids.shape)

X_train_word_ids.shape: (16759,)
X_test_word_ids.shape: (1863,)


# Sequence padding

In [111]:
X_train_padded_seqs = pad_sequences(X_train_word_ids, maxlen=20, value=0)
X_test_padded_seqs = pad_sequences(X_test_word_ids, maxlen=20, value=0)

In [112]:
print('X_train_padded_seqs.shape:', X_train_padded_seqs.shape)
print('X_test_padded_seqs.shape:', X_test_padded_seqs.shape)

X_train_padded_seqs.shape: (16759, 20)
X_test_padded_seqs.shape: (1863, 20)


In [114]:
pd.DataFrame(X_train_padded_seqs).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3134,4717,1911,5074,149,269,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3730,5126,1888,2799,1025,266,5062,2227,0,0,0,0,0,0,0,0,0,0,0,0
2,4037,4585,13,5074,77,6257,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7447,136,7458,4751,7439,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6674,3843,4617,1533,251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
pd.DataFrame(X_test_padded_seqs).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,7380,5261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2154,5451,5074,149,5062,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4535,3944,12,5074,149,6257,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6674,7234,162,1826,4931,2329,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,251,935,3500,251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [116]:
#Convert (y) labels to vectors

unique_labels = list(y_train.value_counts().index)
unique_labels

['Great',
 'Good',
 'Okay',
 'Mediocre',
 'Amazing',
 'Bad',
 'Awful',
 'Painful',
 'Unbearable',
 'Masterpiece']

In [119]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(unique_labels)

LabelEncoder()

In [120]:
for label_id, label_name in zip(le.transform(unique_labels), unique_labels):
    print('%d: %s' %(label_id, label_name))

4: Great
3: Good
7: Okay
6: Mediocre
0: Amazing
2: Bad
1: Awful
8: Painful
9: Unbearable
5: Masterpiece


In [121]:
y_train = to_categorical(y_train.map(lambda x: le.transform([x])[0]), nb_classes=len(unique_labels))
y_test = to_categorical(y_test.map(lambda x: le.transform([x])[0]), nb_classes=len(unique_labels))

In [122]:
y_train[0:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [123]:
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

y_train.shape: (16759, 10)
y_test.shape: (1863, 10)


# Network Building

In [164]:
size_of_each_vector = X_train_padded_seqs.shape[1]
vocab_size = len(vocab)
no_of_unique_y_labels = len(unique_labels)

In [165]:
print('size_of_each_vector:', size_of_each_vector)
print('vocab_size:', vocab_size)
print('no_of_unique_y_labels:', no_of_unique_y_labels)

size_of_each_vector: 20
vocab_size: 7596
no_of_unique_y_labels: 10


In [166]:
net = tflearn.input_data([None, size_of_each_vector]) # The first element is the "batch size" which we set to "None"
net = tflearn.embedding(net, input_dim=vocab_size, output_dim=128) # input_dim: vocabulary size
net = tflearn.lstm(net, 128, dropout=0.6) # Set the dropout to 0.6
net = tflearn.fully_connected(net, no_of_unique_y_labels, activation='softmax') # relu or softmax
net = tflearn.regression(net, 
                         optimizer='adam',  # adam or ada or adagrad # sgd
                         learning_rate=1e-4,
                         loss='categorical_crossentropy')

In [167]:
#initialize the model
model = tflearn.DNN(net, tensorboard_verbose=0)

FailedPreconditionError: Attempting to use uninitialized value is_training
	 [[Node: is_training/read = Identity[T=DT_BOOL, _device="/job:localhost/replica:0/task:0/device:CPU:0"](is_training)]]

Caused by op 'is_training/read', defined at:
  File "C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\snettani\AppData\Local\Con

In [135]:
#Accuracy
import numpy as np
from sklearn import metrics

In [140]:
pred_classes = [np.argmax(i) for i in model.predict(X_test_padded_seqs)]

IndexError: list index out of range

In [143]:
X_test_padded_seqs.shape

(1863, 20)

In [146]:
true_classes = [np.argmax(i) for i in y_test]

In [129]:
print('RNN Accuracy: ' % metrics.accuracy_score(true_classes, pred_classes))

RNN Accuracy: 
